<h1 style="font-size:4rem;color:blue;">Urban Air Pollution Challenge</h1>

<h2 style="font-size:3rem;color:gray;">Problem Statement</h2>


<h1 style="font-size:2rem;color:black;">Predicting the PM2.5 particulate matter concentration in every day for each city. </h1>

### Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from datetime import datetime
from datetime import date
from numpy import mean
from numpy import std

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

import warnings
warnings.filterwarnings('ignore')

In [3]:
# settings

pd.set_option('display.max_columns',None)

### Datasets

In [55]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
sub = pd.read_csv("sub_1.csv")

## Train Dataset

In [5]:
print(train.shape)
train.head()

(30557, 82)


,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,0

In [6]:
print(f'Place_ID : ', str(train['Place_ID'].nunique()), ' Locations in his dataset')
print(f'Place_ID X Date : ', str(train['Place_ID X Date'].nunique()), "entries for this dataset")

Place_ID :  340  Locations in his dataset
Place_ID X Date :  30557 entries for this dataset





There are 340 Locations in this dataset





In [7]:
train.columns

Index(['Place_ID X Date', 'Date', 'Place_ID', 'target', 'target_min',
       'target_max', 'target_variance', 'target_count',
       'precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_NO2_NO2_column_number_density',
       'L3_NO2_NO2_slant_column_number_density',
       'L3_NO2_absorbing_aerosol_index', 'L3_NO2_cloud_fraction',
       'L3_NO2_sensor_altitude', 'L3_NO2_sensor_azimuth_angle',
       'L3_NO2_sensor_zenith_angle', 'L3_NO2_solar_azimuth_angle',
       'L3_NO2_solar_zenith_angle',
       'L3_NO2_stratospheric_NO2_column_number_density',
       'L3_NO2_tropopause_pressure',
       'L3_NO2_tropospheric_NO2_column_number_density',
       'L3_O3_O3_column_number_density', 'L3_O3_O3_effective_temperature',
       'L3_O3_cloud_fraction', 'L3_O3_sensor_azimuth_angle',
   

In [8]:
train.isna().sum()

Place_ID X Date                     0
Date                                0
Place_ID                            0
target                              0
target_min                          0
                                ...  
L3_CH4_aerosol_optical_depth    24765
L3_CH4_sensor_azimuth_angle     24765
L3_CH4_sensor_zenith_angle      24765
L3_CH4_solar_azimuth_angle      24765
L3_CH4_solar_zenith_angle       24765
Length: 82, dtype: int64

In [9]:
#getting columns with count, with having atleast 1 null value

train[train.columns[train.isnull().any()]].isnull().sum()

L3_NO2_NO2_column_number_density           2189
L3_NO2_NO2_slant_column_number_density     2189
L3_NO2_absorbing_aerosol_index             2189
L3_NO2_cloud_fraction                      2189
L3_NO2_sensor_altitude                     2189
                                          ...  
L3_CH4_aerosol_optical_depth              24765
L3_CH4_sensor_azimuth_angle               24765
L3_CH4_sensor_zenith_angle                24765
L3_CH4_solar_azimuth_angle                24765
L3_CH4_solar_zenith_angle                 24765
Length: 68, dtype: int64

In [10]:
#gettingthe percentage of null count

round(train[train.columns[train.isnull().any()]].isnull().sum()*100/train.shape[0]).reset_index(name="Percentage")

,index,Percentage
0,L3_NO2_NO2_column_number_density,7.0
1,L3_NO2_NO2_slant_column_number_density,7.0
2,L3_NO2_absorbing_aerosol_index,7.0
3,L3_NO2_cloud_fraction,7.0
4,L3_NO2_sensor_altitude,7.0
...,...,...
63,L3_CH4_aerosol_optical_depth,81.0
64,L3_CH4_sensor_azimuth_angle,81.0
65,L3_CH4_sensor_zenith_angle,81.0
66,L3_CH4_solar_azimuth_angle,81.0


In [11]:
train_desc = train.describe()
print("There are "+ str(train_desc.shape[1]) + " numerical columns")
train_desc


There are 79 numerical columns


,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
count,30557.000000,30557.000000,30557.000000,3.055700e+04,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,28368.000000,28368.000000,28368.000000,28368.000000,28368.000000,28367.000000,28367.000000,28367.000000,28367.000000,28367.000000,28367.000000,21888.000000,30274.000000,30274.000000,30274.000000,30274.000000,30274.000000,30274.000000,30274.000000,25134.000000,25134.000000,25134.000000,25134.000000,25134.000000,25134.000000,25134.000000,25134.000000,2.313200e+04,23132.000000,23132.000000,23132.000000,23132.000000,23132.000000,23132.000000,23132.000000,29035.000000,29035.000000,30183.000000,29035.000000,29035.000000,29035.000000,30183.000000,30183.000000,30183.000000,30183.000000,29035.000000,30360.000000,30360.000000,30360.000000,30360.000000,30360.000000,30360.000000,23320.000000,23320.000000,23320.000000,23249.000000,23320.000000,23320.000000,23320.000000,23320.000000,23320.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000
mean,61.148045,29.025866,117.992234,7.983756e+03,125.831135,15.302326,70.552747,0.006004,9.321342,0.416886,0.088486,0.000094,0.000163,-1.093982,0.278011,755880.302425,-10.597347,32.764304,-111.413784,44.432617,0.000032,13342.570998,0.000056,0.139533,222.293932,0.423740,-11.308360,35.727367,-117.739509,47.752032,0.036230,983.448217,1438.006332,821611.615359,-12.590815,35.009648,-124.192829,47.605340,4.320685e-05,0.199705,-14.127472,35.849977,-122.640817,46.802819,0.000088,1.064105,2510.989311,77491.774334,0.433513,18.201564,3286.134009,70799.498221,-11.556380,36.520395,-120.055549,48.584734,0.226843,-1.161213,832553.776969,-11.152614,36.667009,-118.855468,48.752878,0.000113,0.429221,0.000032,-1.

In [12]:
#filtering out columns that have more 80% null values
na_80 = [i for i in train.columns if train[i].isnull().sum() > 0.80*len(train)]
na_80

['L3_CH4_CH4_column_volume_mixing_ratio_dry_air',
 'L3_CH4_aerosol_height',
 'L3_CH4_aerosol_optical_depth',
 'L3_CH4_sensor_azimuth_angle',
 'L3_CH4_sensor_zenith_angle',
 'L3_CH4_solar_azimuth_angle',
 'L3_CH4_solar_zenith_angle']

In [13]:
#columns with no missing values
no_na = [col for col in train.columns if not train[col].isnull().any()]
no_na

['Place_ID X Date',
 'Date',
 'Place_ID',
 'target',
 'target_min',
 'target_max',
 'target_variance',
 'target_count',
 'precipitable_water_entire_atmosphere',
 'relative_humidity_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'temperature_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground']

In [14]:
null_columns = [c for c in train.columns if c not in ['Place_ID X Date',
 'Date',
 'Place_ID',
 'target',
'target_min',
 'target_max',
 'target_variance',
 'target_count',                                                       
 'precipitable_water_entire_atmosphere',
 'relative_humidity_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'temperature_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground',
]]

for i in null_columns:
    train[i].fillna(0, inplace=True)

In [15]:
train.isnull().any().sum()

0

In [16]:
#extracting info from date column
#weekend column
#month column
#day column
# quarterly column

train['Date'] = pd.to_datetime(train['Date'], format='%Y-%m-%d')

def applyer(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0      
    
def makedate(df):
    df['dayofweek']=df['Date'].dt.dayofweek
    df['weekend'] = df['Date'].apply(applyer)
    df['month']=0
    df['day']=0
    for i in range(len(df)):  
           
            df['month'][i]=df['Date'][i].month 
            df['day'][i]=df['Date'][i].day
    df["quarter"] = df.Date.dt.quarter        
            
            
makedate(train)                            

In [17]:
train_df = train
train_df.head(2)

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,dayofweek,weekend,month,day,quarter
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.0,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000000,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.00004,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,3,0,1,2,1
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.6,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.00005,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652,4,0,1,3,1


In [18]:
train_df.corr()

,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,dayofweek,weekend,month,day,quarter
target,1.000000,0.818803,0.630801,0.184818,0.094135,-0.031329,-0.084835,-0.035670,-0.038818,-0.148202,-0.055100,0.296010,0.297512,0.008232,-0.105791,0.069198,0.001889,0.049658,-0.174614,0.022844,0.051706,-0.053533,0.261532,-0.054067,0.064139,-0.099922,0.000772,0.023776,-0.192846,-0.034738,0.237537,0.019346,-0.070043,0.047574,0.004711,0.037521,-0.146126,0.011430,0.302243,0.047767,0.002748,0.073545,-0.188222,0.065389,0.322742,0.024251,-0.073154,0.099985,-0.104942,-0.107691,-0.087617,0.121479,0.002856,0.018130,-0.192861,-0.049288,-0.027870,0.067569,-0.001504,0.005920,0.006351,-0.192835,-0.076399,0.046079,0.049365,0.061960,0.007394,0.033368,0.006669,0.068793,-0.184737,0.061320,0.108226,0.107347,0.095703,0.008722,0.081029,-0.119346,0.093706,0.012284,0.001661,-0.122800,0.005238,-0.035266
target_min,0.818803,1.000000,0.421529,0.047010,-0.065336,-0.031739,-0.032409,-0.023465,-0.032910,-0.127322,-0.035046,0.203868,0.240597,-0.000704,-0.075907,0.073891,0.002447,0.049157,-0.146390,0.043877,0.044162,-0.032144,0.223872,-0.039899,0.056180,-0.092148,0.000840,0.018079,-0.150331,-0.006882,0.204102,0.025730,-0.059633,0.057690,0.003562,0.041833,-0.122575,0.033800,0.261827,0.052457,-0.000821,0.068914,-0.157697,0.072366,0.279977,0.035214,-0.067691,0.090964,-0.096853,-0.105300,-0.079908,0.109740,0.002880,0.013299,-0.149674,-0.017133,-0.006823,0.053196,0.010684,0.006145,0.003931,-0.149028,-0.039780,0.031086,0.051671,0.043635,0.004116,0.037409,0.002550,0.062900,-0.154945,0.068893,0.097009,0.096957,0.080255,0.013369,0.071201,-0.102292,0.085082,0.008545,0.000448,-0.119445,0.011662,-0.033487
target_max,0.630801,0.421529,1.000000,0.500524,0.163211,-0.026826,-0.105940,-0.020967,-0.005837,-0.08

In [19]:
#Indentifying columns with using keywords

# simple_feats = [c for c in train_df if ('angle' not in c) & ('height' not in c) & ('altittude' not in c)]

In [20]:
features = ["L3_NO2_NO2_slant_column_number_density","L3_NO2_tropospheric_NO2_column_number_density",  
            "L3_CO_CO_column_number_density", "L3_HCHO_HCHO_slant_column_number_density",
                               "L3_HCHO_tropospheric_HCHO_column_number_density_amf", "L3_CLOUD_cloud_base_pressure",
                                       "L3_AER_AI_absorbing_aerosol_index", "L3_SO2_absorbing_aerosol_index", ]

#### For us to create more patterns that the model will understand, we create more features with previous & next day results, for several features which correlation is above 60% 

In [21]:
#L3_NO2_NO2_slant_column_number_density
for i in range(1,5):
    train_df[f'prev_L3_NO2_NO2_slant_column_number_density_{i}'] = train_df.L3_NO2_NO2_slant_column_number_density.fillna(method='ffill').shift(i)
    train_df[f'next_L3_NO2_NO2_slant_column_number_density_{i}'] = train_df.L3_NO2_NO2_slant_column_number_density.fillna(method='ffill').shift(-i)

#L3_NO2_tropospheric_NO2_column_number_density
for i in range(1,5):
    train_df[f'prev_L3_NO2_tropospheric_NO2_column_number_density_{i}'] = train_df.L3_NO2_tropospheric_NO2_column_number_density.fillna(method='ffill').shift(i)
    train_df[f'next_L3_NO2_tropospheric_NO2_column_number_density_{i}'] = train_df.L3_NO2_tropospheric_NO2_column_number_density.fillna(method='ffill').shift(-i)  
#L3_CO_CO_column_number_density
for i in range(1,5):
    train_df[f'prev_L3_CO_CO_column_number_density_{i}'] = train_df.L3_CO_CO_column_number_density.fillna(method='ffill').shift(i)
    train_df[f'next_L3_CO_CO_column_number_density_{i}'] = train_df.L3_CO_CO_column_number_density.fillna(method='ffill').shift(-i)

#L3_HCHO_HCHO_slant_column_number_density
for i in range(1,5):
    train_df[f'prev_L3_HCHO_HCHO_slant_column_number_density_{i}'] = train_df.L3_HCHO_HCHO_slant_column_number_density.fillna(method='ffill').shift(i)
    train_df[f'next_L3_HCHO_HCHO_slant_column_number_density_{i}'] = train_df.L3_HCHO_HCHO_slant_column_number_density.fillna(method='ffill').shift(-i)

#L3_HCHO_tropospheric_HCHO_column_number_density    
for i in range(1,5):
    train_df[f'prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_{i}'] = train_df.L3_HCHO_tropospheric_HCHO_column_number_density_amf.fillna(method='ffill').shift(i)
    train_df[f'next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_{i}'] = train_df.L3_HCHO_tropospheric_HCHO_column_number_density_amf.fillna(method='ffill').shift(-i)

#L3_CLOUD_cloud_base_pressure    
for i in range(1,5):
    train_df[f'prev_L3_CLOUD_cloud_base_pressure_{i}'] = train_df.L3_CLOUD_cloud_base_pressure.fillna(method='ffill').shift(i)
    train_df[f'next_L3_CLOUD_cloud_base_pressure_{i}'] = train_df.L3_CLOUD_cloud_base_pressure.fillna(method='ffill').shift(-i)
    
# L3_AER_AI_absorbing_aerosol_index  
for i in range(1,5):
    train_df[f'prev_L3_AER_AI_absorbing_aerosol_index_{i}'] = train_df.L3_AER_AI_absorbing_aerosol_index.fillna(method='ffill').shift(i)
    train_df[f'next_L3_AER_AI_absorbing_aerosol_index_{i}'] = train_df.L3_AER_AI_absorbing_aerosol_index.fillna(method='ffill').shift(-i)   
#L3_SO2_absorbing_aerosol_index 
for i in range(1,5):
    train_df[f'prev_L3_SO2_absorbing_aerosol_index_{i}'] = train_df.L3_SO2_absorbing_aerosol_index.fillna(method='ffill').shift(i)
    train_df[f'next_L3_SO2_absorbing_aerosol_index_{i}'] = train_df.L3_SO2_absorbing_aerosol_index.fillna(method='ffill').shift(-i)   
# precipitable_water_entire_atmosphere   
for i in range(1,5):
    train_df[f'prev_precipitable_water_entire_atmosphere_{i}'] = train_df.precipitable_water_entire_atmosphere.fillna(method='ffill').shift(i)
    train_df[f'next_precipitable_water_entire_atmosphere_{i}'] = train_df.precipitable_water_entire_atmosphere.fillna(method='ffill').shift(-i)
#specific_humidity_2m_above_ground    
for i in range(1,5):
    train_df[f'prev_specific_humidity_2m_above_ground_{i}'] = train_df.specific_humidity_2m_above_ground.fillna(method='ffill').shift(i)    
    train_df[f'next_specific_humidity_2m_above_ground_{i}'] = train_df.specific_humidity_2m_above_ground.fillna(method='ffill').shift(-i)
#temperature_2m_above_ground    
for i in range(1,5):
    train_df[f'prev_temperature_2m_above_ground_{i}'] = train_df.temperature_2m_above_ground.fillna(method='ffill').shift(i)
    train_df[f'next_temperature_2m_above_ground_{i}'] = train_df.temperature_2m_above_ground.fillna(method='ffill').shift(-i)

In [22]:
print(train_df.shape)
train_df.head(3)

(30557, 175)


,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,dayofweek,weekend,month,day,quarter,prev_L3_NO2_NO2_slant_column_number_density_1,next_L3_NO2_NO2_slant_column_number_density_1,prev_L3_NO2_NO2_slant_column_number_density_2,next_L3_NO2_NO2_slant_column_number_density_2,prev_L3_NO2_NO2_slant_column_number_density_3,next_L3_NO2_NO2_slant_column_number_density_3,prev_L3_NO2_NO2_slant_column_number_density_4,next_L3_NO2_NO2_slant_column_number_density_4,prev_L3_NO2_tropospheric_NO2_column_number_density_1,next_L3_NO2_tropospheric_NO2_column_number_density_1,prev_L3_NO2_tropospheric_NO2_column_number_density_2,next_L3_NO2_tropospheric_NO2_column_number_density_2,prev_L3_NO2_tropospheric_NO2_column_number_density_3,next_L3_NO2_tropospheric_NO2_column_number_density_3,prev_L3_NO2_tropospheric_NO2_column_number_density_4,next_L3_NO2_tropospheric_NO2_column_number_density_4,prev_L3_CO_CO_column_number_density_1,next_L3_CO_CO_column_number_density_1,prev_L3_CO_CO_column_number_density_2,next_L3_CO_CO_column_number_density_2,prev_L3_CO_CO_column_number_density_3,next_L3_CO_CO_column_number_density_3,prev_L3_CO_CO_column_number_density_4,next_L3_CO_CO_column_number_density_4,prev_L3_HCHO_HCHO_slant_column_number_density_1,next_L3_HCHO_HCHO_slant_column_number_density_1,prev_L3_HCHO_HCHO_slant_column_number_density_2,next_L3_HCHO_HCHO_slant_column_number_density_2,prev_L3_HCHO_HCHO_slant_column_number_density_3,next_L3_HCHO_HCHO_slant_column_number_density_3,prev_L3_HCHO_HCHO_slant_column_number_density_4,next_L3_HCHO_HCHO_slant_column_number_density_4,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_1,next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_1,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_2,next_L

In [23]:
train_df.fillna(0, inplace=True)
train_df.isna().any().sum()

0

In [24]:
# dropping features columns
# dropping columns with more 80% missing values

train_df_cols = train_df.drop(features, axis=1)
train_df_cols = train_df_cols.drop(na_80, axis=1)

In [25]:
train_df_cols.shape

(30557, 160)

In [26]:
train_df_cols.corr()

,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,dayofweek,weekend,month,day,quarter,prev_L3_NO2_NO2_slant_column_number_density_1,next_L3_NO2_NO2_slant_column_number_density_1,prev_L3_NO2_NO2_slant_column_number_density_2,next_L3_NO2_NO2_slant_column_number_density_2,prev_L3_NO2_NO2_slant_column_number_density_3,next_L3_NO2_NO2_slant_column_number_density_3,prev_L3_NO2_NO2_slant_column_number_density_4,next_L3_NO2_NO2_slant_column_number_density_4,prev_L3_NO2_tropospheric_NO2_column_number_density_1,next_L3_NO2_tropospheric_NO2_column_number_density_1,prev_L3_NO2_tropospheric_NO2_column_number_density_2,next_L3_NO2_tropospheric_NO2_column_number_density_2,prev_L3_NO2_tropospheric_NO2_column_number_density_3,next_L3_NO2_tropospheric_NO2_column_number_density_3,prev_L3_NO2_tropospheric_NO2_column_number_density_4,next_L3_NO2_tropospheric_NO2_column_number_density_4,prev_L3_CO_CO_column_number_density_1,next_L3_CO_CO_column_number_density_1,prev_L3_CO_CO_column_number_density_2,next_L3_CO_CO_column_number_density_2,prev_L3_CO_CO_column_number_density_3,next_L3_CO_CO_column_number_density_3,prev_L3_CO_CO_column_number_density_4,next_L3_CO_CO_column_number_density_4,prev_L3_HCHO_HCHO_slant_column_number_density_1,next_L3_HCHO_HCHO_slant_column_number_density_1,prev_L3_HCHO_HCHO_slant_column_number_density_2,next_L3_HCHO_HCHO_slant_column_number_density_2,prev_L3_HCHO_HCHO_slant_column_number_density_3,next_L3_HCHO_HCHO_slant_column_number_density_3,prev_L3_HCHO_HCHO_slant_column_number_density_4,next_L3_HCHO_HCHO_slant_column_number_density_4,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_1,next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_1,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_2,next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_2,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_3,next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_3,prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_4,next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_4,prev_L3_CLOUD_cloud_base_pressure_1,next_L3_CLOUD_cloud_base_pressure_1,prev_L3_CLOUD_cloud_base_pressure_2,next_L3_CLOUD_cloud_base_pressure_2,prev_L3_CLOUD_cloud_base_pressure_3,next_L3_CLOUD_cloud_base_pressure_3,prev_L3_CLOUD_cloud_base_pressure

In [27]:
# Filtering columns with less than 0.1 correlation

In [28]:
threshold = 0.1
corr = pd.Series(train_df_cols.corrwith(train_df["target"]) > threshold) 

train_df_cor = train_df_cols[corr[corr == True].index] 

In [29]:
train_df_cor.shape

(30557, 43)

In [30]:
train_df_cor = train_df_cor.drop(labels = ["target_min","target_max","target_variance"], axis=1)

In [31]:
print(train_df_cor.shape)
train_df_cor.head()

(30557, 40)


,target,L3_NO2_NO2_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density,L3_CLOUD_cloud_top_pressure,prev_L3_NO2_NO2_slant_column_number_density_1,next_L3_NO2_NO2_slant_column_number_density_1,prev_L3_NO2_NO2_slant_column_number_density_2,next_L3_NO2_NO2_slant_column_number_density_2,prev_L3_NO2_NO2_slant_column_number_density_3,next_L3_NO2_NO2_slant_column_number_density_3,prev_L3_NO2_NO2_slant_column_number_density_4,next_L3_NO2_NO2_slant_column_number_density_4,prev_L3_NO2_tropospheric_NO2_column_number_density_1,next_L3_NO2_tropospheric_NO2_column_number_density_1,prev_L3_NO2_tropospheric_NO2_column_number_density_2,next_L3_NO2_tropospheric_NO2_column_number_density_2,prev_L3_NO2_tropospheric_NO2_column_number_density_3,next_L3_NO2_tropospheric_NO2_column_number_density_3,prev_L3_NO2_tropospheric_NO2_column_number_density_4,next_L3_NO2_tropospheric_NO2_column_number_density_4,prev_L3_CO_CO_column_number_density_1,next_L3_CO_CO_column_number_density_1,prev_L3_CO_CO_column_number_density_2,next_L3_CO_CO_column_number_density_2,prev_L3_CO_CO_column_number_density_3,next_L3_CO_CO_column_number_density_3,prev_L3_CO_CO_column_number_density_4,next_L3_CO_CO_column_number_density_4,prev_L3_HCHO_HCHO_slant_column_number_density_1,next_L3_HCHO_HCHO_slant_column_number_density_1,prev_L3_HCHO_HCHO_slant_column_number_density_2,next_L3_HCHO_HCHO_slant_column_number_density_2,prev_L3_HCHO_HCHO_slant_column_number_density_3,next_L3_HCHO_HCHO_slant_column_number_density_3,prev_L3_HCHO_HCHO_slant_column_number_density_4,next_L3_HCHO_HCHO_slant_column_number_density_4,prev_L3_CLOUD_cloud_base_pressure_1,prev_L3_CLOUD_cloud_base_pressure_2,prev_L3_CLOUD_cloud_base_pressure_3,prev_L3_CLOUD_cloud_base_pressure_4
0,38.0,0.000074,0.000064,0.000000,0.000000,0.000197,0.000000,0.000170,0.000000,0.000175,0.000000,0.000142,0.000000,0.000021,0.000000,0.000007,0.000000,0.000023,0.000000,0.000009,0.000000,0.022017,0.000000,0.020677,0.000000,0.021207,0.00000,0.037766,0.000000,0.000114,0.000000,0.000027,0.000000,2.348694e-05,0.00000,3.724964e-05,0.000000,0.000000,0.000000,0.0
1,39.0,0.000076,0.000171,99353.672374,0.000156,0.000170,0.000000,0.000175,0.000000,0.000142,0.000000,0.000146,0.000017,0.000007,0.000000,0.000023,0.000000,0.000009,0.000000,0.000012,0.021080,0.020677,0.000000,0.021207,0.000000,0.037766,0.00000,0.029486,-0.000010,0.000027,0.000000,0.000023,0.000000,3.724964e-05,0.00000,9.163087e-06,0.000000,0.000000,0.000000,0.0
2,24.0,0.000067,0.000124,95671.383578,0.000197,0.000175,0.000156,0.000142,0.000000,0.000146,0.000000,0.000179,0.000021,0.000023,0.000017,0.000009,0.000000,0.000012,0.000000,0.000000,0.022017,0.021207,0.021080,0.037766,0.000000,0.029486,0.00000,0.036858,0.000114,0.000023,-0.000010,0.000037,0.000000,9.163087e-06,0.00000,-1.304491e-04,99354.162958,0.000000,0.000000,0.0
3,49.0,0.000083,0.000081,96232.486418,0.000170,0.000142,0.000197,0.000146,0.000156,0.000179,0.000000,0.000199,0.000007,0.000009,0.000021,0.000012,0.000017,0.000000,0.000000,0.000008,0.020677,0.037766,0.022017,0.029486,0.021080,0.036858,0.00000,0.035093,0.000027,0.000037,0.000114,0.000009,-0.000010,-1.304491e-04,0.00000,7.562204e-07,98118.935248,99354.162958,0.000000,0.0
4,21.0,0.000070,0.000140,52561.523079,0.000175,0.000146,0.000170,0.000179,0.000197,0.000199,0.000156,0.000156,0.000023,0.000012,0.000007,0.000000,0.000021,0.000008,0.000017,0.000013,0.021207,0.029486,0.020677,0.036858,0.022017,0.035093,0.02108,0.033269,0.000023,0.000009,0.000027,-0.000130,0.000114,7.562204e-07,-0.00001,3.230138e-05,97258.462755,98118.935248,99354.162958,0.0


In [32]:
# Calculate the correlation matrix
correlation_matrix = train_df_cor.corr()

# Select the features with high correlation values
high_correlation_features = correlation_matrix[correlation_matrix > threshold].index

# Keep only the selected features in the dataframe
selected_features_df = train_df[high_correlation_features]

In [33]:
target = train['target']

# Specify the number of top features to select
k = 10

# Instantiate the feature selector
feature_selector = SelectKBest(score_func=f_regression, k=k)

# Fit the feature selector to the data
feature_selector.fit(selected_features_df, target)

# Get the indices of the selected features
selected_feature_indices = feature_selector.get_support(indices=True)

# Get the names of the selected features
selected_features = selected_features_df.columns[selected_feature_indices]

# Print the selected feature names
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['target', 'L3_NO2_NO2_column_number_density',
       'L3_HCHO_tropospheric_HCHO_column_number_density',
       'prev_L3_NO2_NO2_slant_column_number_density_1',
       'prev_L3_NO2_NO2_slant_column_number_density_2',
       'prev_L3_NO2_tropospheric_NO2_column_number_density_1',
       'prev_L3_HCHO_HCHO_slant_column_number_density_1',
       'next_L3_HCHO_HCHO_slant_column_number_density_1',
       'prev_L3_HCHO_HCHO_slant_column_number_density_2',
       'next_L3_HCHO_HCHO_slant_column_number_density_2'],
      dtype='object')


In [34]:
final_train = train_df_cor[selected_features]

In [35]:
final_train.shape

(30557, 10)

In [36]:
final_train.head(3)

,target,L3_NO2_NO2_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density,prev_L3_NO2_NO2_slant_column_number_density_1,prev_L3_NO2_NO2_slant_column_number_density_2,prev_L3_NO2_tropospheric_NO2_column_number_density_1,prev_L3_HCHO_HCHO_slant_column_number_density_1,next_L3_HCHO_HCHO_slant_column_number_density_1,prev_L3_HCHO_HCHO_slant_column_number_density_2,next_L3_HCHO_HCHO_slant_column_number_density_2
0,38.0,0.000074,0.000064,0.000000,0.000000,0.000000,0.000000,0.000114,0.00000,0.000027
1,39.0,0.000076,0.000171,0.000156,0.000000,0.000017,-0.000010,0.000027,0.00000,0.000023
2,24.0,0.000067,0.000124,0.000197,0.000156,0.000021,0.000114,0.000023,-0.00001,0.000037


## Test Dataset

In [37]:
print(test.shape)
test.head(3)

(16136, 77)


,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,0.000108,0.466171,0.010752,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000046,6156.074219,0.000007,0.113310,227.467539,0.032071,68.099367,1.445658,-95.984984,22.942019,0.017953,841.142869,155.982981,835625.785337,32.709708,1.898112,-95.987015,22.950150,0.000073,0.032071,68.099367,1.445658,-95.984984,22.942019,0.000088,1.494039,2592.634167,74028.429228,0.032268,6.317523,2593.137433,74023.930722,68.099367,1.445658,-95.984984,22.942019,0.299053,0.466173,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000221,0.784436,0.000184,-0.140458,0.032071,68.099367,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,0.000109,-0.213659,0.028307,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000045,7311.869141,0.000005,0.110397,227.788713,0.044784,75.936845,34.639165,-95.017596,18.539491,0.019576,1187.570320,922.385833,835225.278332,73.808143,34.720869,-95.017095,18.548551,0.000048,0.040803,75.936813,34.641758,-95.014908,18.539116,0.000074,1.534485,7334.601102,48466.653300,0.046211,8.308433,7902.937546,46345.613031,75.936845,34.639165,-95.017596,18.539491,0.290397,-0.213657,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000034,0.678988,0.000014,-0.842713,0.040803,75.936813,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0OS9LVX X 2020-01-04,2020-01-04,0OS9LVX,17.600000,41.299999,0.00590,15.511041,5.245728,1.640559,0.000050,0.000134,-0.254250,0.010374,834839.050781,75.552445,55.872276,-94.015418,14.140820,0.000044,6156.074219,0.000006,0.112502,229.235631,0.007113,75.552445,55.8

In [52]:
test['Date'] = pd.to_datetime(test['Date'], format='%Y-%m-%d')
makedate(test)                            

In [53]:
print(test.shape)
test.head(2)

(16136, 82)


,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,dayofweek,weekend,month,day,quarter
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,0.000108,0.466171,0.010752,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000046,6156.074219,0.000007,0.113310,227.467539,0.032071,68.099367,1.445658,-95.984984,22.942019,0.017953,841.142869,155.982981,835625.785337,32.709708,1.898112,-95.987015,22.950150,0.000073,0.032071,68.099367,1.445658,-95.984984,22.942019,0.000088,1.494039,2592.634167,74028.429228,0.032268,6.317523,2593.137433,74023.930722,68.099367,1.445658,-95.984984,22.942019,0.299053,0.466173,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000221,0.784436,0.000184,-0.140458,0.032071,68.099367,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,1,2,1
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,0.000109,-0.213659,0.028307,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000045,7311.869141,0.000005,0.110397,227.788713,0.044784,75.936845,34.639165,-95.017596,18.539491,0.019576,1187.570320,922.385833,835225.278332,73.808143,34.720869,-95.017095,18.548551,0.000048,0.040803,75.936813,34.641758,-95.014908,18.539116,0.000074,1.534485,7334.601102,48466.653300,0.046211,8.308433,7902.937546,46345.613031,75.936845,34.639165,-95.017596,18.539491,0.290397,-0.213657,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000034,0.678988,0.000014,-0.842713,0.040803,75.936813,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,1,3,1


In [56]:
def feature_eng(df):
    #L3_NO2_NO2_slant_column_number_density
    for i in range(1,5):
        test_df[f'prev_L3_NO2_NO2_slant_column_number_density_{i}'] = test_df.L3_NO2_NO2_slant_column_number_density.fillna(method='ffill').shift(i)
        test_df[f'next_L3_NO2_NO2_slant_column_number_density_{i}'] = test_df.L3_NO2_NO2_slant_column_number_density.fillna(method='ffill').shift(-i)

#L3_NO2_tropospheric_NO2_column_number_density
    for i in range(1,5):
        test_df[f'prev_L3_NO2_tropospheric_NO2_column_number_density_{i}'] = test_df.L3_NO2_tropospheric_NO2_column_number_density.fillna(method='ffill').shift(i)
        test_df[f'next_L3_NO2_tropospheric_NO2_column_number_density_{i}'] = test_df.L3_NO2_tropospheric_NO2_column_number_density.fillna(method='ffill').shift(-i)  
#L3_CO_CO_column_number_density
    for i in range(1,5):
        test_df[f'prev_L3_CO_CO_column_number_density_{i}'] = test_df.L3_CO_CO_column_number_density.fillna(method='ffill').shift(i)
        test_df[f'next_L3_CO_CO_column_number_density_{i}'] = test_df.L3_CO_CO_column_number_density.fillna(method='ffill').shift(-i)

#L3_HCHO_HCHO_slant_column_number_density
    for i in range(1,5):
        test_df[f'prev_L3_HCHO_HCHO_slant_column_number_density_{i}'] = test_df.L3_HCHO_HCHO_slant_column_number_density.fillna(method='ffill').shift(i)
        test_df[f'next_L3_HCHO_HCHO_slant_column_number_density_{i}'] = test_df.L3_HCHO_HCHO_slant_column_number_density.fillna(method='ffill').shift(-i)

#L3_HCHO_tropospheric_HCHO_column_number_density    
    for i in range(1,5):
        test_df[f'prev_L3_HCHO_tropospheric_HCHO_column_number_density_amf_{i}'] = test_df.L3_HCHO_tropospheric_HCHO_column_number_density_amf.fillna(method='ffill').shift(i)
        test_df[f'next_L3_HCHO_tropospheric_HCHO_column_number_density_amf_{i}'] = test_df.L3_HCHO_tropospheric_HCHO_column_number_density_amf.fillna(method='ffill').shift(-i)

#L3_CLOUD_cloud_base_pressure    
    for i in range(1,5):
        test_df[f'prev_L3_CLOUD_cloud_base_pressure_{i}'] = test_df.L3_CLOUD_cloud_base_pressure.fillna(method='ffill').shift(i)
        test_df[f'next_L3_CLOUD_cloud_base_pressure_{i}'] = test_df.L3_CLOUD_cloud_base_pressure.fillna(method='ffill').shift(-i)
    
# L3_AER_AI_absorbing_aerosol_index  
    for i in range(1,5):
        test_df[f'prev_L3_AER_AI_absorbing_aerosol_index_{i}'] = test_df.L3_AER_AI_absorbing_aerosol_index.fillna(method='ffill').shift(i)
        test_df[f'next_L3_AER_AI_absorbing_aerosol_index_{i}'] = test_df.L3_AER_AI_absorbing_aerosol_index.fillna(method='ffill').shift(-i)   
#L3_SO2_absorbing_aerosol_index 
    for i in range(1,5):
        test_df[f'prev_L3_SO2_absorbing_aerosol_index_{i}'] = test_df.L3_SO2_absorbing_aerosol_index.fillna(method='ffill').shift(i)
        test_df[f'next_L3_SO2_absorbing_aerosol_index_{i}'] = test_df.L3_SO2_absorbing_aerosol_index.fillna(method='ffill').shift(-i)   
# precipitable_water_entire_atmosphere   
    for i in range(1,5):
        test_df[f'prev_precipitable_water_entire_atmosphere_{i}'] = test_df.precipitable_water_entire_atmosphere.fillna(method='ffill').shift(i)
        test_df[f'next_precipitable_water_entire_atmosphere_{i}'] = test_df.precipitable_water_entire_atmosphere.fillna(method='ffill').shift(-i)
#specific_humidity_2m_above_ground    
    for i in range(1,5):
        test_df[f'prev_specific_humidity_2m_above_ground_{i}'] = test_df.specific_humidity_2m_above_ground.fillna(method='ffill').shift(i)    
        test_df[f'next_specific_humidity_2m_above_ground_{i}'] = test_df.specific_humidity_2m_above_ground.fillna(method='ffill').shift(-i)
#temperature_2m_above_ground    
    for i in range(1,5):
        test_df[f'prev_temperature_2m_above_ground_{i}'] = test_df.temperature_2m_above_ground.fillna(method='ffill').shift(i)
        test_df[f'next_temperature_2m_above_ground_{i}'] = test_df.temperature_2m_above_ground.fillna(method='ffill').shift(-i)
        
        return test_df
feature_eng(test)

AttributeError: 'NoneType' object has no attribute 'L3_NO2_NO2_slant_column_number_density'

In [45]:
print(test_df.shape)
test_df.head(2)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def null(df):
    df.fillna(0, inplace=True)
    train_df_cols = df.drop(features, axis=1)
    train_df_cols = train_df_cols.drop(na_80, axis=1)
    
    return train_df_cols
test_df_cor = null(test_df)

In [ ]:
print(test_df_cor.shape)
test_df_cor.head(2)

In [ ]:
#dropping less important columns

common_cols=list(set.intersection(set(train_df_cor.columns), set(test_df_cor.columns)))
drop_cols = [k for k in test_df_cor.columns if k not in common_cols]
test_df_cols = test_df_cor.drop(drop_cols, axis=1)

In [ ]:
test_df_cols.shape

In [ ]:
         **************

In [ ]:
# after feature selection
test_features = final_train.drop('target', axis=1).columns
final_test = test_df_cols[test_features]

In [ ]:
       ****************

In [ ]:
z_test_cols = np.abs(stats.zscore(final_test))

In [ ]:
# Random scaling

standard_scaler = StandardScaler()
for k in z_test_cols.columns:
    standard_scaler.fit_transform(z_test_cols[[k]])

In [ ]:
print(z_test_cols.shape)
z_test_cols.head()

## Train Test Split

In [ ]:
X = final_train.drop('target', axis=1)

y = final_train["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,  random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#Z score
Xtrain_z = np.abs(stats.zscore(X_train))
Xtest_z = np.abs(stats.zscore(X_test))

In [ ]:
# Random scaling

standard_scaler = StandardScaler()
for k in Xtrain_z.columns:
    standard_scaler.fit_transform(Xtrain_z[[k]])
    
standard_scaler = StandardScaler()
for k in Xtest_z.columns:
    standard_scaler.fit_transform(Xtest_z[[k]]) 

In [ ]:
Xtrain_z.head()

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
# Instantiate model
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

# Fit the model
model.fit(Xtrain_z,y_train)

In [ ]:
rf = model.predict(z_test_cols)

In [ ]:
predicts = test[['Place_ID X Date']]
predicts['target'] = rf
predicts.to_csv('rf_sub.csv', index=False)

In [ ]:
y_pred = model.predict(Xtest_z)
np.sqrt(mse(y_test, y_pred))

In [ ]:
# model score
model.score(Xtrain_z,y_train)

In [ ]:
# RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV


# Different RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"]}

# Instantiate RandomizedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=2,
                              cv=2,
                              verbose=True)

# Fit the RandomizedSearchCV model
rs_model.fit(Xtrain_z,y_train)

In [ ]:
y_pred = rs_model.predict(Xtest_z)
np.sqrt(mse(y_test, y_pred))

In [ ]:
rs_model.best_params_

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV


# Different RandomForestRegressor hyperparameters
rf_grid = {'n_estimators': [90],
            'min_samples_split': [14],
            'max_features': ['sqrt'],
            'max_depth': [None]}

# Instantiate RandomizedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=2,
                              cv=2,
                              verbose=True)

# Fit the RandomizedSearchCV model
rs_model.fit(Xtrain_z,y_train)
model.score(Xtrain_z,y_train)

In [ ]:

from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(Xtrain_z,y_train,
             use_best_model=True,
             verbose=50)


In [ ]:
cb_pred2 = cb_model.predict(z_test_cols)

In [ ]:
y_pred = cb_model.predict(Xtest_z)
np.sqrt(mse(y_test, y_pred))

In [ ]:
# Find feature importance of our best model
rs_model.feature_importances_

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV
params = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5,
               0.6,0.7, 0.8, 0.9]}]
print("starting grid search ......")
optimized_GBM = GridSearchCV(LGBMRegressor(),
                             params,
                             cv=3,
                             n_jobs=-1)
# 
optimized_GBM.fit(Xtrain_z,y_train)
#preds2 = optimized_GBM.predict(te)

In [ ]:
y_pred = optimized_GBM.predict(Xtest_z)
np.sqrt(mse(y_test, y_pred))